Archivo de Jupyter para cálculo simbólico de elementos de matriz

In [1]:
# Computación numérica
import numpy as np
from scipy.special import erf
# Computación simbólica
from sympy import symbols, simplify, integrate, oo, E
from sympy.vector import CoordSys3D, laplacian

In [2]:
A = CoordSys3D('A', transformation='spherical') # Sistema coordenado esférico, centrado en A
B = CoordSys3D('B', transformation='spherical') # Sistema coordenado esférico, centrado en B

a1, a2, a3 = symbols('a1 a2 a3', positive=True) # exponentes orbitales Gaussianos
d1, d2, d3 = symbols('d1 d2 d3', real=True) # coeficientes de contracción

En general un elemento de matriz $f_{pq}$ será dado por
$$
f_{pq} = \int \chi_p (\hat{f}_1 + \hat{f}_2) \chi_q \, \mathrm{d}\mathbf{x}
    = \int \psi_p (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r} \int Y_{00}^2 \, \mathrm{d}\Omega \int \sigma_p(\omega) \sigma_q(\omega) \, \mathrm{d}\omega
$$

La parte angular es 1 y la parte espinorial se trata de una delta de Kronecker, por lo cual solo los elementos de matriz $f_{pq}$ tal que sus partes espinoriales coincidan serán diferentes de cero.

Desarrollando la parte radial donde $\mathrm{d}\mathbf{r} = r^2$ es
$$
\begin{align}
\int \psi_p (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r}
    &= \int (\varphi_A \pm_p \varphi_B) (\hat{f}_1 + \hat{f}_2) (\varphi_A \pm_q \varphi_B) \, \mathrm{d}\mathbf{r} \\
    &= (\varphi_A | \hat{f}_1 | \varphi_A) \pm_p (\varphi_B | \hat{f}_1 | \varphi_A) \pm_q (\varphi_A | \hat{f}_1 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_1 | \varphi_B)
    + (\varphi_A | \hat{f}_2 | \varphi_A) \pm_p (\varphi_B | \hat{f}_2 | \varphi_A) \pm_q (\varphi_A | \hat{f}_2 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_2 | \varphi_B)
\end{align}
$$

Aquí $\varphi_C$ se trata de una $k$-contracción Gaussiana $1\mathrm{s}$ centrada en $C$ de la forma:

$$ \varphi_C = \sum_{i=1}^{k} d_i \phi_i^{(C)} $$

donde $\phi_i^{(C)} = \exp(-\alpha_i |\mathbf{r} - \mathbf{R}_C|^2) = \exp(-\alpha_i r_C^2)$ es una Gaussiana $1\mathrm{s}$ no normalizada. Es importante hacer notar esto pues la constante de normalización para las Gaussianas $1\mathrm{s}$ dependen únicamente del exponente orbital $\alpha_i$ y por tanto serán añadidas hasta el final del cálculo.

In [3]:
def const(a):
    """
        Constante de normalización de una función tipo Gaussiana 1s
    a : exponente orbital Gaussiano
    """
    return 2*np.power(2*a,3/4)/np.power(np.pi,1/4)

Por simplicidad, se empleará la convención de notación $f_{i}^{(CD)} = (\varphi_C | \hat{f}_i | \varphi_D)$ donde $i \in \{1,2\}$ y $C, D \in \{A,B\}$. Esto para cada uno de los términos del elemento de matriz. En cuyo caso, el elemento de matriz $f_{pq}$ luciría como
$$
\begin{align}
    f_{pq} &=
    f_{1}^{(AA)} \pm_p f_{1}^{(BA)} \pm_q f_{1}^{(AB)} \pm_p \pm_q f_{1}^{(BB)}
    + f_{2}^{(AA)} \pm_p f_{2}^{(BA)} \pm_q f_{2}^{(AB)} \pm_p \pm_q f_{2}^{(BB)}
\end{align}
$$

Recuerde que cada función $\varphi_C$ se trata de una $k$-contracción Gaussiana $1\mathrm{s}$ centrada en $C$. Más aún, note que las funciones $\varphi_A$ y $\varphi_B$ comparten el mismo conjunto de parámetros Gaussianos $\{ \mathbf{d}, \mathbf{a} \}$ y su única diferencia es estar centradas en núcleos distintos. Sin embargo, matemáticamente el estar centrada en otro núcleo no es más que un etiquetado, i.e. $A$ y $B$ son mudas. Por lo tanto, $f_{1}^{(AA)} = f_{1}^{(BB)}$, $f_{1}^{(AB)} = f_{1}^{(BA)}$, $f_{2}^{(AA)} = f_{2}^{(BB)}$ y $f_{2}^{(AB)} = f_{2}^{(BA)}$, así pues el elemento de matriz $f_{pq}$ será:

$$ f_{pq} = 2\delta_{pq} \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

Entonces los elementos de matriz fuera de la diagonal $f_{pq}$, aunque $p$ y $q$ sean ambos pares o nones, son cero. Es decir, solo los elementos en la diagonal son diferentes de cero.

$$ f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

<div class='alert alert-block alert-info'><strong>Término $f_{1}^{(AA)}$</strong></div>

El término $f_{1}^{(AA)}$ puede ser integrado simbólicamente con facilidad y evaluado de la misma manera.

$$
f_{1}^{AA}
= (\varphi_A | \hat{f}_1 | \varphi_A) = \int r^2 \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(A)} \right) \mathrm{d}r
= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(A)} \mathrm{d}r
$$

Para los términos $\hat{f}_1 \phi_{j}^{(A)}$ se tiene que

$$
\hat{f}_1 \phi_{j}^{(A)} = \left( -\frac{1}{2} \nabla^2 \right) \exp(-\alpha_j r^2)
= -\frac{1}{2} \frac{1}{r^2} \frac{\partial}{\partial r} \left( r^2 \frac{\partial}{\partial r}\exp\left[-\alpha_j r^2 \right] \right)
= \alpha_j (3 - 2 \alpha_j r^2) \exp(-\alpha_j r^2)
= \alpha_j (3 - 2 \alpha_j r^2) \phi_{j}^{(A)}
$$

In [4]:
# corroboración trabajo previo
lap = simplify((-1/2) * laplacian(E**(-a2*(A.r**2))))
lap

a2*(-2.0*A.r**2*a2 + 3.0)*exp(-A.r**2*a2)

In [5]:
ind_f1AA = simplify((A.r**2) * E**(-a1*(A.r**2)) * lap) # integrando
int_f1AA = simplify(integrate(ind_f1AA, (A.r,0,oo))) # integral
int_f1AA
# Nota: Al integrando se añade el elemento radial r^2 pues SymPy no lo añade a pesar que A.r es una coordenada radial

0.75*sqrt(pi)*a1*a2/(a1 + a2)**(5/2)

De este modo, el elemento de matriz $f_1^{AA}$ será dado por
$$
f_{1}^{AA} = \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \alpha_j(3 - 2 \, \alpha_j \, r^2) \phi_{j}^{(A)} \mathrm{d}r
= \frac{3}{4} \sqrt{\pi} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{5/2}}
$$

In [6]:
# término f_1^AA del elemento de matriz f_pq, cómputo directo

def f1AA_term(di:float, dj:float, ai:float, aj:float):
    """
        Término f_1^AA
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    """
    return di*dj*ai*aj/np.power(ai+aj,5/2)
    
def f1AA(d, a):
    """
        Término f_1^AA. Suma de funciones base
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    """
    k = len(d)
    suma = 0
    for i in range(k):
        for j in range(k):
            # const(a) = const. de normalización de Gaussianas 1s
            suma += const(a[i]) * const(a[j]) * f1AA_term(d[i], d[j], a[i], a[j])
    return (3/4)*np.sqrt(np.pi)*suma

In [7]:
print("Se fijan todos los parámetros igual a 1.")
print("Cómputo directo:  ", f1AA([1],[1]))
print("Cómputo simbólico:", const(1)*const(1)*int_f1AA.subs({a1:1, a2:1}).evalf())

Se fijan todos los parámetros igual a 1.
Cómputo directo:   1.4999999999999998
Cómputo simbólico: 1.50000000000000


<div class='alert alert-block alert-info'><strong>Término $f_{1}^{(AB)}$</strong></div>

Observe que $f_{1}^{(AB)}$ constará de $k^2$ términos pues
$$
\begin{align}
f_{1}^{(AB)} &= (\varphi_A | \hat{f}_1 | \varphi_B)
= \int r^2 \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}r
= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(B)} \, \mathrm{d}r
\end{align}
$$

Esta última integral tiene solución y es de la forma [2, pág. 412]:
$$
\begin{align}
    f_{1}^{(AB)}
    &= \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} \left[ \frac{3 - 2\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha_i + \alpha_j} \right]^{3/2} \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right] \\
    &= \pi^{3/2} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{7/2}} (3 - 2\alpha_i \alpha_j) |\mathbf{R}_A - \mathbf{R}_B|^2 \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right]
\end{align}
$$

la expresión general para $f_{1}^{(AB)}$.

In [8]:
def f1AB_term(di:float, dj:float, ai:float, aj:float, RA, RB):
    """
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    """
    R = np.linalg.norm(np.array(RA)-np.array(RB), ord=1) # distancia interatómica al cuadrado
    return di*dj*ai*aj/np.power(ai+aj, 7/2)*(3-2*ai*aj)*R*np.exp(-ai*aj*R/(ai+aj))

def f1AB(d, a, RA, RB):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    R : vector de núcleos [Ra, Rb] donde Rc = [C1, C2, C3]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    """
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            # const(a) = const. de normalización de Gaussianas 1s
            suma += const(a[i]) * const(a[j]) * f1AB_term(d[i], d[j], a[i], a[j], RA, RB)
    return np.power(np.pi, 3/2) * suma

<div class='alert alert-block alert-info'><strong>Término $f_{2}^{(AA)}$</strong></div>

Para $f_{2}^{(AA)}$ se tiene

$$
\begin{align}
f_{2}^{(AA)} = (\varphi_A | \hat{f}_2 | \varphi_A)
&= \int r^2_A \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_2 \left( \sum_j^{k} d_j \phi_{j}^{(A)} \right) \, \mathrm{d}r_A\\
&= \sum_i^{k} \sum_j^{k} d_i d_j \int r_A^2 \phi_{i}^{(A)}  \left( - \sum_C \frac{Z_C}{r_C} \right) \phi_{j}^{(A)} \, \mathrm{d}r_A\\
&= \sum_i^{k} \sum_j^{k} d_i d_j \left( - \sum_C Z_C \int r_A^2 \phi_{i}^{(A)} \frac{1}{r_C} \phi_{j}^{(A)} \, \mathrm{d}r_A \right)
\end{align}
$$

donde $r_A = |\mathbf{r} - \mathbf{R}_A|$ y $r_C = |\mathbf{r} - \mathbf{R}_C|$ con $\mathbf{r}$ la coordenada del electrón en cuestión y $\mathbf{R}_A$, $\mathbf{R}_C$ del núcleo $A$, $C$ respectivamente.

Se busca escribir a la $r_C$ en términos de $r_A$ y de $R_{AC} = |\mathbf{R}_A - \mathbf{R}_C|$, la distancia entre los núcleos $A$ y $C$. Por la ley de cosenos es posible hacer esto, tendiendo así

$$ r_C^2 = r_A^2 + R_{AC}^2 - 2 r_A R_{AC} \cos\theta $$

donde $\theta$ es el ángulo entre el vector $\mathbf{r} - \mathbf{R}_A$ y el vector $\mathbf{R}_{A} - \mathbf{R}_B$. Así la integral de interés por resolver es

$$ \int r_A^2 \phi_{i}^{(A)} \frac{1}{r_C} \phi_{j}^{(A)} \, \mathrm{d}r_A
= \int r_A^2 \frac{\exp[-(\alpha_i + \alpha_j) r_A^2]}{r_C} \, \mathrm{d}r_A
$$

<div class='alert alert-block alert-warning'>
    1. Se tendría que integrar sobre la coordenada angular? (en cuyo caso hay que introducir el elemento $\Omega$ que no estábamos considerando ya) <br>
    2. ¿Se podría calcular numéricamente?
</div>

<div class='alert alert-block alert-info'><strong>Término $f_{2}^{(AB)}$</strong></div>

Para $f_{2}^{(AB)}$ se tiene
$$
\begin{align}
    f_{2}^{(AB)} = (\varphi_A | \hat{f}_2 | \varphi_B)
    &= \int \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \left( - \sum_C \frac{Z_C}{r_C} \right) \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}\mathbf{r}\\
    &= \sum_i^{k} \sum_j^{k} d_i d_j \sum_C \left\{ \int \phi_{i}^{(A)} \left( - \frac{Z_C}{r_C} \right) \phi_{j}^{(B)} \, \mathrm{d}\mathbf{r} \right\} \\
    &= \sum_i^{k} \sum_j^{k} d_i d_j \sum_C \left\{ -\frac{2\pi}{\alpha_i + \alpha_j} Z_C \, \exp\left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    F_0 \left[ (\alpha_i + \alpha_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \right] \right\} \\
    &= -2\pi \sum_i^{k} \sum_j^{k} \left\{ \frac{d_i d_j}{\alpha_i + \alpha_j} \exp\left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    \sum_C Z_C \,
    F_0 \left[ (\alpha_i + \alpha_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \right] \right\}
\end{align}
$$

donde $C \in \{A, B\}$ es alguno de los dos núcleos posibles, $\mathbf{R}_P$ es un punto en la línea que une a los centros $A$ y $B$ dado por
$$
\begin{align}
    \mathrm{R}_P = \frac{\alpha_i \mathbf{R}_A + \alpha_j \, \mathbf{R}_B}{\alpha_i + \alpha_j}
\end{align}
$$

y $F_0$ es la función dada por
$$
\begin{align}
    F_0 (t) = \frac{1}{2} \sqrt{\frac{\pi}{t}} \mathrm{erf} (\sqrt{t})
\end{align}
$$

con $\mathrm{erf}$ la función de error.

In [9]:
def Rp(a, b, RA, RB):
    return (a*np.array(RA) + b*np.array(RB))/(a+b)

def F0(t):
    return (1/2)*np.sqrt(np.pi/t) * erf(np.sqrt(t))

In [10]:
def f2AB_term(di:float, dj:float, ai:float, aj:float, R, Z):
    """
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    Ra : coord. núcleo A, [A1, A2, A3]
    Rb : coord. núcleo B, [B1, B2, B3]
    Za : carga nuclear núcleo A
    Zb : carga nuclear núcleo B
    """
    # len(R) = len(Z) = 2
    Rab = np.linalg.norm(np.array(R[0])-np.array(R[1]), ord=1) # distancia interatómica al cuadrado, |RA-RB|^2
    RP = Rp(ai, aj, R[0], R[1])
    
    suma = 0
    for C in range(2):
        Rpc = np.linalg.norm(RP - R[C], ord=1) # distancia al cuadrado entre el centro P y C |RP-RC|^2
        suma += di*dj / (ai+aj) * np.exp(-ai*aj*Rab/(ai+aj)) * Z[C] * F0((ai+aj)*Rpc)
    return suma

def f2AB(d, a, RA, RB, ZA, ZB):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    ZA : carga nuclear núcleo A
    ZB : carga nuclear núcleo B
    """
    # len(d) = len(a) = k
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            # const(a) = const. de normalización de Gaussianas 1s
            suma += const(a[i]) * const(a[j]) * f2AB_term(d[i], d[j], a[i], a[j], [RA, RB], [ZA, ZB])
    return -2*np.pi * suma

<div class='alert alert-block alert-success'><strong>Elemento de matriz $f_{pp}$</strong></div>

Finalmente, es posible calcular el elemento de matriz $f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right]$. Si $p=1,2$ entonces el signo es positivo, mientras que si $p=3,4$ el signo será negativo.

In [11]:
def fpp(p, d, a, RA, RB, ZA, ZB):
    """
        Elemento de matriz f_pp
    p : p-ésimo elemento de la base ordenada
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    ZA : carga nuclear núcleo A
    ZB : carga nuclear núcleo B
    """
    term1AA = f1AA(d, a)
    term1AB = f1AB(d, a, RA, RB)
    term2AA = f2AB(d, a, RA, RA, ZA, ZB)
    term2AB = f2AB(d, a, RA, RB, ZA, ZB)

    if p in [1,2]:
        return 2*(term1AA + term1AB + term2AA + term2AB)
    else:
        return 2*(term1AA - term1AB + term2AA - term2AB)

In [12]:
# Parámetros Gaussianos obtenidos en `(1) STO-3G.ipynb`
d = [0.44471812476789035, 0.5352716544572346, 0.1543000507808527]
a = [0.10983311305458726, 0.40584959263313375, 2.227949029647934]

RA = [0, 0, 0]
RB = [1.401, 0, 0]
ZA = 1
ZB = 1

f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)

C:\Users\rodri\AppData\Local\Temp\ipykernel_13580\2150372359.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  return (1/2)*np.sqrt(np.pi/t) * erf(np.sqrt(t))
C:\Users\rodri\AppData\Local\Temp\ipykernel_13580\2150372359.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  return (1/2)*np.sqrt(np.pi/t) * erf(np.sqrt(t))


In [13]:
f1AA(d, a)

np.float64(0.49430511544236067)

In [14]:
f1AB(d, a, RA, RB)

np.float64(7.701115573801517)

In [15]:
f2AB(d, a, RA, RB, ZA, ZB)

np.float64(-15.727157569842069)

In [16]:
f1AA(d, a) + f1AB(d, a, RA, RB) + f2AB(d, a, RA, RB, ZA, ZB)

np.float64(-7.53173688059819)